In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
from LUloader import LUloader


In [ ]:
%matplotlib inline
Func_types = ['Restrict', 'U', 'C', 'T']
Func_cmap = ['white', 'lightgray', 'springgreen', 'forestgreen']

CL = LUloader(LUs=Func_types, asProb=True, LUType='V', LUname='lu', LUcol='lu', driver='feather', cmap=Func_cmap, path='CL')
CL.plot_LU()

In [ ]:
w = pd.read_csv('CL/Ws.csv').set_index('PAC')
wx = gpd.read_feather('CL/lu.feather')
wx['grid_code'] = wx['grid_code'].astype(int)
gdp = w.GDP
p = pd.DataFrame({i: wx.grid_code == i for i in gdp.index}).astype(int)

wx['food'] = wx['RASTERVALU'] * wx['recultivat'] / 10 * wx['grid_code'].apply(lambda x: w['FOOD'][x] if x not in [710000, 810000] else 0)
wx['tree'] = wx['RASTERVALU'] * wx['reforest'] / 10 * wx['grid_code'].apply(lambda x: w['NEP'][x] if x not in [710000, 810000] else 0)
wx['biod'] = wx['RASTERVALU'] * wx['reforest'] / 10 * wx['grid_code'].apply(lambda x: w['BIO'][x] if x not in [710000, 810000] else 0)

wx['foodvaluec'] = wx['RASTERVALU'] * wx['recultivat'] / 10 * wx['grid_code'].apply(lambda x: w['FOODVALUEC'][x] if x not in [710000, 810000] else 0)
wx['foodvalued'] = wx['RASTERVALU'] * wx['recultivat'] / 10 * wx['grid_code'].apply(lambda x: w['FOODVALUED'][x] if x not in [710000, 810000] else 0)

wx['treevaluec'] = wx['RASTERVALU'] * wx['reforest'] / 10 * wx['grid_code'].apply(lambda x: w['TREEVALUEC'][x] if x not in [710000, 810000] else 0)
wx['treevalued'] = wx['RASTERVALU'] * wx['reforest'] / 10 * wx['grid_code'].apply(lambda x: w['TREEVALUED'][x] if x not in [710000, 810000] else 0)


wx = wx[['pointid', 'food', 'tree', 'biod', 'foodvaluec', 'foodvalued', 'treevaluec', 'treevalued', 'RASTERVALU']]

maskR = np.ones_like(CL.var.LU)
maskR[:, 1] = wx['food'] > 0
maskR[:, 2] = wx['tree'] > 0
maskR[:, 3] = wx['biod'] > 0
maskR[:, 4] = wx['foodvaluec'] > 0
maskR[:, 5] = wx['foodvalued'] > 0
maskR[:, 6] = wx['treevaluec'] > 0
maskR[:, 7] = wx['treevalued'] > 0




CCM = np.loadtxt('CL/CCM.csv', delimiter=',')
CCM = 1 - CCM / CCM.max()

In [ ]:
M = {'LU': CL}
con = dict(food=wx['food'].to_numpy(), tree=wx['tree'].to_numpy(), biod=wx['biod'].to_numpy(), foodvaluec=wx['foodvaluec'].to_numpy(), foodvalued=wx['foodvalued'].to_numpy(), treevaluec=wx['treevaluec'].to_numpy(), treevalued=wx['treevalued'].to_numpy(), CCM=CCM, oLU=CL.var.LU.to_numpy().copy())

In [ ]:
from model import GDAmodel
import objs

thC = 0.98  # threshod of conversion limit
Model = GDAmodel(M, con, maskR, objs, thC=thC, GPU=1, path='CL')
# print(Model.LU.shape)

Model.init_train(500, lr=0.1)       # no init file: run init_train
Model.save_inp()                    # save init file
# Model.init_train_load('inp', 0.1)  # skip init_train and load init file
Model.log()

In [ ]:
from model import GDAmodel
import objs

thC = 0.98  # threshod of conversion limit
Model = GDAmodel(M, con, maskR, objs, thC=thC, GPU=1, path='CL')
Model.init_train_load('inp', 0.1)  # skip init_train and load init file


In [ ]:
Model.perference([1, 1, 1, 1, 1])
Model.train(n_iter=1000, thStop=1, nprint=50)  # set nprint=None to close printing and ploting
# img will be saved in plot/LU


In [ ]:
Model.log()
# Model.plot_opt()

In [ ]:
print(Model.get_opt()['LU'].argmax(axis=1))
print('food', np.sum(Model.get_opt()['LU'].argmax(axis=1)==1), 'tree', np.sum(Model.get_opt()['LU'].argmax(axis=1)==2))

In [65]:
lu_matrix = Model.get_opt()['LU']

# 将矩阵转换为 DataFrame
df = pd.DataFrame(lu_matrix, columns=['Column1', 'Column2', 'Column3'])
excel_file_path = 'duo4output.xlsx'
with pd.ExcelWriter(excel_file_path) as writer:
    df.to_excel(writer, index=False, sheet_name='Sheet1')

In [66]:
gdf = gpd.read_file('duo/4.shp')
gdf['LU'] = Model.get_opt()['LU'].argmax(axis=1)
gdf.to_file('duo/4-1.shp')